In [63]:
# Importiamo le librerie necessarie
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, DoubleType, TimestampType
from pyspark.sql.functions import col, year, sum, to_timestamp, count, expr, max, count_distinct, expr, when, avg, min, month, floor, lag
from pyspark.sql import Window
from pyspark.sql.dataframe import DataFrame
import numpy as np

# df_updated = df.withColumns({
#     "Country": when((col("Country") == "America") & (col("Population") > 10000), "North America").otherwise(col("Country")),
#     "Region": when((col("Country") == "America") & (col("Population") > 10000), "NA").otherwise(col("Region"))
# })

# df_updated = df.withColumn(
#     "Country",
#     when(col("Country") == "America", 
#          when(col("Population") > 10000, "North America")
#          .when(col("Population") > 5000, "Central America")
#          .otherwise("South America"))
#     .otherwise(col("Country"))
# )

# df_updated = df.withColumn(
#     "Country",
#     expr("CASE WHEN Country = 'America' AND Population > 10000 THEN 'North America' ELSE Country END")
# )

# CASE 
#     WHEN Country = 'America' AND Population > 10000 THEN 'North America' 
#     WHEN Country = 'America' AND Population > 5000 THEN 'Central America'
#     WHEN Country = 'America' THEN 'South America' 
#     ELSE Country
# END

# response2 = (
#     monthly_water_consumption
#     .withColumn("Year", year(col("Month")))
#     .groupBy(col("HID"), col("Year"))
#     .agg(sum("M3").alias("AnnualM3"))
#     .withColumn("PreviousAnnualM3", lag("AnnualM3").over(
#         Window
#         .partitionBy("HID")
#         .orderBy(col("Year"))
#     ))
#     .filter(col("PreviousAnnualM3") > col("AnnualM3")
# )
    


# Supponiamo che SparkSession sia già stato creato
ss: SparkSession = SparkSession.builder.appName("PoliSalesAnalysis").getOrCreate()

# Variabili per i percorsi di input e output
# Percorsi dei file di input e output
jupyter = False
if jupyter:
    input_prefix = "/user/s339450/esami/20240912/"
    output_prefix= "/user/s339450/esami/20240912/out/"
else:
    input_prefix = ".\\data\\"
    output_prefix= ".\\out\\"

houses_path = f"{input_prefix}Houses.txt"
monthly_water_consumption_path = f"{input_prefix}MonthlyWaterConsumption.txt"
output_folder_1 = f"{output_prefix}1/"
output_folder_2 = f"{output_prefix}2/"

In [64]:
houses_schema = StructType([
    StructField("HID", StringType(), False),
    StructField("City", StringType(), False),
    StructField("Country", StringType(), False),
    StructField("YearBuilt", IntegerType(), False)
])

houses: DataFrame = ss.read.load(houses_path,
    format="csv",
    header=False,
    schema=houses_schema,
    sep=",")

houses.show()

monthly_water_consumption_schema = StructType([
    StructField("HID", StringType(), False),
    StructField("Month", StringType(), False),
    StructField("M3", IntegerType(), False)
])

monthly_water_consumption: DataFrame = ss.read.load(monthly_water_consumption_path,
    format="csv",
    header=False,
    schema=monthly_water_consumption_schema,
    sep=",")

# Conversione del timestamp
monthly_water_consumption = monthly_water_consumption.withColumn(
    "Month",
    to_timestamp(col("Month"), "yyyy/MM")  # Adatta il formato del timestamp
)

monthly_water_consumption.show()

+-----+---------+-----------+---------+
|  HID|     City|    Country|YearBuilt|
+-----+---------+-----------+---------+
|H3402|    Turin|      Italy|     1965|
|H5821|    Paris|     France|     1978|
|H1290|   Berlin|    Germany|     1990|
|H8765|   London|         UK|     2005|
|H4321|   Madrid|      Spain|     1983|
|H7654|     Rome|      Italy|     1970|
|H9087|Amsterdam|Netherlands|     2015|
|H2345| New York|        USA|     1988|
|H6789|   Sydney|  Australia|     2000|
|H3456|    Tokyo|      Japan|     1995|
+-----+---------+-----------+---------+

+-----+-------------------+---+
|  HID|              Month| M3|
+-----+-------------------+---+
|H3402|2021-01-01 00:00:00|  8|
|H3402|2021-04-01 00:00:00|  9|
|H3402|2021-07-01 00:00:00|  7|
|H3402|2021-10-01 00:00:00|  6|
|H3402|2022-01-01 00:00:00| 12|
|H3402|2022-04-01 00:00:00| 13|
|H3402|2022-07-01 00:00:00| 10|
|H3402|2022-10-01 00:00:00|  9|
|H5821|2021-01-01 00:00:00| 10|
|H5821|2021-04-01 00:00:00| 12|
|H5821|2021-07-01 00:00

# Punto 1

In [65]:
# trimester_converter = withColumn(
#     "Trimester_2022",
#     col=(
#         when(month(col("Month")) <= 3, 1)
#         .when(((month(col("Month")) > 3) & (month(col("Month")) <= 6)), 2)
#         .when(((month(col("Month")) > 6) & (month(col("Month")) <= 9)), 3)
#         .otherwise(4)
#     )
# )

monthly_water_consumption_2022 = (
    monthly_water_consumption
    .filter(year(col("Month")) == 2022)
    .withColumn(colName="Trimester_2022", col=(floor(month((col("Month"))) / 4) + 1))
    .groupBy("HID", "Trimester_2022")
    .agg(sum("M3"))
    .withColumnRenamed("sum(M3)", "M3_2022")
    .alias("monthly_water_consumption_2022")
)

# monthly_water_consumption_2022.show()

monthly_water_consumption_2021 = (
    monthly_water_consumption
    .filter(year(col("Month")) == 2021)
    .withColumn(colName="Trimester_2021", col=(floor(month((col("Month"))) / 4) + 1))
    .groupBy("HID", "Trimester_2021")
    .agg(sum("M3"))
    .withColumnRenamed("sum(M3)", "M3_2021")
    .alias("monthly_water_consumption_2021")
)

# monthly_water_consumption_2021.show()


response1 = (
    monthly_water_consumption_2022
    .join(
        other=monthly_water_consumption_2021,
        on=(
            (col("Trimester_2022") == col("Trimester_2021"))
            & (col("monthly_water_consumption_2022.HID") == col("monthly_water_consumption_2021.HID"))
        )
    )
    .filter(col("M3_2022") > col("M3_2021"))
    .groupBy(col("monthly_water_consumption_2022.HID"))
    .agg(count("*"))
    .filter("count(1) >= 3")
    .select("HID")
)

response1.show()

+-----+
|  HID|
+-----+
|H8765|
|H1290|
|H5821|
|H3402|
+-----+



# Punto 2

In [69]:
response2 = (
    monthly_water_consumption
    .withColumn("Year", year(col("Month")))
    .groupBy(col("HID"), col("Year"))
    .agg(sum("M3").alias("AnnualM3"))
    .withColumn("PreviousAnnualM3", lag("AnnualM3").over(
        Window
        .partitionBy("HID")
        .orderBy(col("Year"))
    ))
    .filter(col("PreviousAnnualM3") > col("AnnualM3"))
    .orderBy("HID", "Year")
)

response2.show()

+-----+----+--------+----------------+
|  HID|Year|AnnualM3|PreviousAnnualM3|
+-----+----+--------+----------------+
|H2345|2022|      21|              60|
+-----+----+--------+----------------+

